In [2]:
import pandas as pd
import numpy as np
from functools import reduce
import os
import matplotlib.backends 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
refs        ='/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data/reference/Homo_sapiens.GRCh38.97.chr.txt'
ref         = pd.read_csv(refs, sep="\t", header=0,skipinitialspace=True)
lncRNAs_ref = ref.query('biotype == "lncRNA"')

In [4]:
# gencode except for the nagative ones 12869
prediction_triplexfpp_genocode_all = pd.read_csv('/home/alva/Documents/lncRNAs_project_2020/scripts_2021/TriplexFPP/output/triplexgencode.v36.lncRNA_transcripts_results.tsv',header=None,sep='\t',names=['headers','score','triplex_helices'])
prediction_triplexfpp_genocode_all = prediction_triplexfpp_genocode_all.join(prediction_triplexfpp_genocode_all['headers'].apply(lambda x: pd.Series(str(x).split('|'))).rename(columns=lambda x: f"string_{x+1}"))
prediction_triplexfpp_genocode_all.rename(columns={'string_5':'transcript_sym','string_6':'gene_sym_y'},inplace =True)
triplexfpp_genocode_all_filtered   =prediction_triplexfpp_genocode_all[["gene_sym_y","transcript_sym","triplex_helices","score"]]

In [5]:
tripleFPP_triplex_genecode   = triplexfpp_genocode_all_filtered.query('triplex_helices == "Triplex"')
tripleFPP_triplex_genecode.sort_values('score',ascending=False)

,gene_sym_y,transcript_sym,triplex_helices,score
12072,CR381653.1,CR381653.1-204,Triplex,1.151553
12070,CR381653.1,CR381653.1-202,Triplex,1.148390
7674,MIR100HG,MIR100HG-220,Triplex,1.129234
12028,AL355803.1,AL355803.1-201,Triplex,1.128435
10483,AC008105.3,AC008105.3-204,Triplex,1.111797
...,...,...,...,...
2351,ZMYND10-AS1,ZMYND10-AS1-201,Triplex,0.492086
3912,AC025188.1,AC025188.1-201,Triplex,0.491564
4070,LINC01184,LINC01184-205,Triplex,0.491360
4793,TAB2-AS1,TAB2-AS1-202,Triplex,0.490859


In [6]:
tripleFPP_triplex_genecode.to_csv('/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/DNA_methy_tcga/meth/de/tripleFPP_triplex_genecode', sep='\t',index=True)

In [142]:
tripleFPP_triplex_genecode.query('gene_sym_y == "PVT1"')

,gene_sym_y,transcript_sym,triplex_helices,score
6061,PVT1,PVT1-346,Triplex,0.688675
6062,PVT1,PVT1-316,Triplex,0.647226
6063,PVT1,PVT1-234,Triplex,0.747205
6064,PVT1,PVT1-315,Triplex,0.654007
6065,PVT1,PVT1-304,Triplex,0.670471
...,...,...,...,...
6161,PVT1,PVT1-277,Triplex,0.597450
6162,PVT1,PVT1-361,Triplex,0.602169
6163,PVT1,PVT1-211,Triplex,0.688601
6164,PVT1,PVT1-375,Triplex,0.656961


In [130]:
tripleFPP_triplex_genecode.gene_sym_y.drop_duplicates().shape

(363,)

In [101]:
DE_brca_lncRNAs = pd.read_csv('/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data/lncRNA_0.58_BRCA_DE',header=0,sep='\t')
positve_lncRNAs_triplexFPP = pd.read_csv('/home/alva/Documents/lncRNAs_project_2020/scripts_2021/TriplexFPP_data/data/triplexlncRNA_pos_lncRNAs',names=['gene_sym_y'],sep='\t')

In [136]:
dfs      = [DE_brca_lncRNAs,positve_lncRNAs_triplexFPP,tripleFPP_triplex_genecode]
df_final = reduce(lambda left,right: pd.merge(left,right,on='gene_sym_y'), dfs)
df_final.gene_sym_x.drop_duplicates()

0      SLX1A-SULT1A3
5             HOTAIR
10              PVT1
115        SSTR5-AS1
116       AL121832.2
117       AC069281.2
118          MIR22HG
132       MIRLET7BHG
135             MEG3
184       AL162586.1
185         MIR100HG
273           DIO3OS
275         TBX2-AS1
276           PICSAR
Name: gene_sym_x, dtype: object

In [ ]:
dfs      = [DE_brca_lncRNAs,positve_lncRNAs_triplexFPP,tripleFPP_triplex_genecode]
df_final = reduce(lambda left,right: pd.merge(left,right,on='gene_sym_y'), dfs)
df_final.gene_sym_x.drop_duplicates()

In [143]:
de_brca_positive = pd.merge(DE_brca_lncRNAs,positve_lncRNAs_triplexFPP,on='gene_sym_y')
de_brca_positive_triplexgencode = pd.merge(DE_brca_lncRNAs,tripleFPP_triplex_genecode,on='gene_sym_y')
de_brca_positive_triplexgencode.to_csv('/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data/de_lncRNAs_with_triplex/gencode_triplex_predictions_brca_de',index=False)
